In [13]:
import torch

from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.test_functions import Hartmann
from gpytorch.mlls import ExactMarginalLogLikelihood

neg_hartmann6 = Hartmann(dim=6, negate=True)

In [14]:
train_x = torch.rand(10, 6)
train_x.shape

torch.Size([10, 6])

In [15]:
train_obj = neg_hartmann6(train_x).unsqueeze(-1)
train_obj.shape

torch.Size([10, 1])

In [16]:
model = SingleTaskGP(train_X=train_x, train_Y=train_obj)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll);

/Users/liuqianyi/miniconda3/envs/r244/lib/python3.10/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)
/Users/liuqianyi/miniconda3/envs/r244/lib/python3.10/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([0.2868]), std = tensor([0.2775])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


In [6]:
from botorch.acquisition import ExpectedImprovement

best_value = train_obj.max()
best_value

tensor(0.9895)

In [7]:
EI = ExpectedImprovement(model=model, best_f=best_value)

In [8]:
from botorch.optim import optimize_acqf

new_point_analytic, _ = optimize_acqf(
    acq_function=EI, 
    bounds=torch.tensor([[0.0] * 6, [1.0] * 6]), 
    q=1, 
    num_restarts=20, 
    raw_samples=100,
    options={},
)

In [9]:
new_point_analytic

tensor([[0.3248, 0.2103, 0.7305, 0.3289, 0.4148, 0.5686]])

In [17]:
from botorch.acquisition import qExpectedImprovement
from botorch.sampling import SobolQMCNormalSampler

sampler = SobolQMCNormalSampler(sample_shape=torch.Size([512]), seed=0)
MC_EI = qExpectedImprovement(model, best_f=best_value, sampler=sampler)
torch.manual_seed(seed=0)
new_point_mc, _ = optimize_acqf(
    acq_function=MC_EI,
    bounds=torch.tensor([[0.0] * 6, [1.0] * 6]),
    q=1,
    num_restarts=20,
    raw_samples=100,
    options={},
)

In [38]:
posterior = model.posterior(train_x)

In [39]:
posterior.mean

tensor([[0.2196],
        [0.1021],
        [0.2108],
        [0.0867],
        [0.1958],
        [0.0951],
        [0.0856],
        [0.1180],
        [0.3337],
        [0.1201]], grad_fn=<UnsqueezeBackward0>)

In [33]:
posterior.distribution

MultivariateNormal(loc: torch.Size([10]))

In [40]:
samples = sampler(posterior)

In [43]:
samples.shape # we are making multiple samples of functions, these functions are represented as values at these 512 data points

torch.Size([512, 10, 1])

In [12]:
torch.norm(new_point_mc - new_point_analytic)

tensor(0.0004)